In [4]:
#no transformers used in the end
#build vocabs for Shakespeare plays
#trained to generate shakespeare play based on data, using LSTM
#currently used just for Hamlet. But could be used for any play, or all the plays
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

In [5]:
#These, dear onlooker, are the other models we need to import. Their true fate and purpose shall be revealed in time
import re
import operator
import numpy as np
import seaborn as sns
import pandas as pd
import pickle as pickle #data processing
from collections import Counter #tokenization
import keras #ML
#import processing as pr #helper
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from keras.preprocessing.text import Tokenizer
import torch
from torch import nn
import torch.nn.functional as F
import gensim, logging
from gensim.models import word2vec
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [6]:
shakespeare_db = pd.read_csv("shakespeare-plays/Shakespeare_data.csv")

In [5]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(shakespeare_db['PlayerLine'])

In [6]:
uniques = vectorizer.get_feature_names()

In [7]:
word_array = X.toarray()

In [8]:
def play_string(play):
    string = ""
    for i in play:
        string += i
        string += " "
    return string

In [9]:
def alt_keywords(docs, k):
    lines = [line.rstrip() for line in open('SmartStoplist.txt')]
    stop_list = set(lines)
    vectorizer = TfidfVectorizer(stop_words=stop_list)
    tfidf_matrix = vectorizer.fit_transform(docs)
    tfidf_scores = np.sum(tfidf_matrix,axis=0)
    tfidf_scores = np.ravel(tfidf_scores)
    return sorted(dict(zip(vectorizer.get_feature_names(),tfidf_scores)).items(),key=lambda x:x[1], reverse=True)[:k]

In [10]:
def histogram(words):
    #words_list = re.split("(?:(?:[^a-zA-Z]+')|(?:'[^a-zA-Z]+))|(?:[^a-zA-Z']+)", words)
    words_list = re.split("\W*[^\'\w+\']", words)
    #thanks to Martijn Pieters for the above split
    word_dictionary = {}
    counter = len(words_list)
    i = 0
    while i < counter:
        word_dictionary[words_list[i]] = 1
        n = i+1
        while n < counter:
            if words_list[i] == words_list[n]:
                word_dictionary[words_list[i]] += 1
                words_list.pop(n)
                counter -= 1
                n -= 1
            n += 1
        i += 1
    return word_dictionary
def unique_words(histogram):
    return len(histogram)
def frequency(word, histogram):
    return histogram[word]
def create_sorted_array(histogram):
    new_array = []
    count = 0
    for i in histogram:
        new_array.append([histogram[i]])
        new_array[count].append(i)
        count += 1
    new_array.sort()
    return new_array

In [11]:
def find_alt_key_words(play_name, num):
    play = shakespeare_db[shakespeare_db['Play']==play_name]
    play = play['PlayerLine']
    alt_keys = alt_keywords(play,num)
    return alt_keys

In [12]:
def find_key_words(play_name):
    play = shakespeare_db[shakespeare_db['Play']==play_name]
    play = play['PlayerLine']
    keys = keywords(play,10)
    return keys

In [13]:
def all_the_plays():
    new_play = "Henry IV"
    play_array = []
    for i in shakespeare_db['Play']:
        if i != new_play:
            play_array.append(new_play)
            new_play = i
    return play_array

In [14]:
all_plays = all_the_plays()

In [15]:
play_keywords = [[]] 

In [16]:
play_groups = []

In [17]:
def find_keys():
    for i in all_plays:
        play_words = []
        play_words.append(i)
        play_words.append(find_alt_key_words(i, 50))
        play_keywords.append(play_words)

In [18]:
find_keys()

/usr/local/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['1971', 'ain', 'aren', 'cornell', 'couldn', 'cs', 'didn', 'doesn', 'don', 'english', 'ftp', 'hadn', 'hasn', 'haven', 'isn', 'list', 'mon', 'pub', 'salton', 'shouldn', 'smart', 'stop', 've', 'wasn', 'weren', 'won', 'word', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/usr/local/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['1971', 'ain', 'aren', 'cornell', 'couldn', 'cs', 'didn', 'doesn', 'don', 'english', 'ftp', 'hadn', 'hasn', 'haven', 'isn', 'list', 'mon', 'pub', 'salton', 'shouldn', 'smart', 'stop', 've', 'wasn', 'weren', 'won', 'word', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/usr/local/lib/python3.7

/usr/local/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['1971', 'ain', 'aren', 'cornell', 'couldn', 'cs', 'didn', 'doesn', 'don', 'english', 'ftp', 'hadn', 'hasn', 'haven', 'isn', 'list', 'mon', 'pub', 'salton', 'shouldn', 'smart', 'stop', 've', 'wasn', 'weren', 'won', 'word', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/usr/local/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['1971', 'ain', 'aren', 'cornell', 'couldn', 'cs', 'didn', 'doesn', 'don', 'english', 'ftp', 'hadn', 'hasn', 'haven', 'isn', 'list', 'mon', 'pub', 'salton', 'shouldn', 'smart', 'stop', 've', 'wasn', 'weren', 'won', 'word', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/usr/local/lib/python3.7

In [19]:
def find_groups(keys):
    for i in keys:
        play_thing = [[]]
        for n in i:
            if (play_thing == None) and (i[0] != None):
                play_thing.append([i[0]])
            if n != i[0]:
                for y in n:
                    for x in keys:
                        play_match_words = []
                        for xn in x:
                            if xn != n:
                                if xn != x[0]:
                                    for xy in xn:
                                        if xy[0] == y[0]:
                                            play_match_words.append(xy[0])
                            if (play_match_words != None):
                                play_match = [[x[0]]]
                                play_match[0].append(play_match_words)
                                play_thing[0].append(play_match)
        play_groups.append(play_thing)
                                            

In [20]:
find_groups(play_keywords)

In [21]:
def word_groups(keys):
    word_array = []
    for i in keys:
        for n in i:
            if n != i[0]:
                for y in n:
                    word_in_array = False
                    for w in word_array:
                        if w[0] == y[0]:
                            w.append(i[0])
                            word_in_array = True
                    if word_in_array != True:
                        single_word = [y[0]]
                        single_word.append(i[0])
                        word_array.append(single_word)
    return word_array

In [22]:
w = word_groups(play_keywords)

In [23]:
def play_groups(keys):
    play_array = []
    for i in keys:
        array_within_array = []
        for n in i:
            if n == i[0]:
                array_within_array.append(n)
            else:
                for y in n:
                    for z in w:
                        if y[0] == z[0]:
                            for a in z:
                                repeat = False
                                for b in array_within_array:
                                    if a == b[0]:
                                        repeat = True
                                        b[1] += 1
                                if (repeat != True) and (a != y[0]) and (a != i[0]):
                                    array_within_array.append([a,1])
        play_array.append(array_within_array)
        
    return play_array

In [24]:
p = play_groups(play_keywords)

In [25]:
def grouping_plays(play_group, num):
    group_array = []
    for i in play_group:
        a = []
        for x in i:
            if type(x[1]) == int:
                if x[1] > num:
                    if a == []:
                        a.append(i[0])
                    a.append(x[0])
        if a != []:
            group_array.append(a)
    return group_array

In [26]:
key_words = []

In [27]:
def find_all_keys():
    all_plays = shakespeare_db['PlayerLine']
    keys = keywords(all_plays,5)
    return keys

In [9]:
def get_text():
    h = shakespeare_db[shakespeare_db['Play']=='Hamlet']
    w = h['PlayerLine']
    text = ""
    for i in w:
        text += i
        text += " "
    return text

In [10]:
text = get_text()

In [13]:
text=text.lower()
characters = sorted(list(set(text)))
n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

In [14]:
X = []
Y = []
length = len(text)
seq_length = 100
for i in range(0, length-seq_length, 1):
 sequence = text[i:i + seq_length]
 label =text[i + seq_length]
 X.append([char_to_n[char] for char in sequence])
 Y.append(char_to_n[label])

In [15]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

In [16]:
#model thanks to pranjal52
model = Sequential()
model.add(LSTM(400, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(400))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

Instructions for updating:
Colocations handled automatically by placer.


2019-12-13 01:45:17,188 : WARNING : From /usr/local/lib/python3.7/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


2019-12-13 01:45:18,095 : WARNING : From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [17]:
model.fit(X_modified, Y_modified, epochs=1, batch_size=1000)

model.save_weights('text_generator_400_0.2_400_0.2_baseline.h5')

Instructions for updating:
Use tf.cast instead.


2019-12-13 01:45:29,808 : WARNING : From /usr/local/lib/python3.7/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Epoch 1/1
163707/163707 [==============================] - 6575s 40ms/step - loss: 3.0014


In [18]:
model.load_weights('text_generator_400_0.2_400_0.2_baseline.h5')

In [26]:
#code thanks to pranjal52
string_mapped = X[500]
full_string = [n_to_char[value] for value in string_mapped]
# generating characters
for i in range(400):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(characters))

    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [n_to_char[value] for value in string_mapped]
    full_string.append(n_to_char[pred_index])

    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]

In [27]:
#combining text, generating text in the style of Shakespeare
txt=""
for char in full_string:
    txt = txt+char
txt

"e. i think i hear them. stand, ho! who's there? enter horatio and marcellus friends to this ground.                                                                                                                                                                                                                                                                                                                                                                                                                 "